## Q5: Planetary orbits

We want to consider planetary orbits.  To do this, we need to solve Newton's second law together with Newton's law of gravity.  If we restrict ourselves to the x-y plane, then there are 4 quantities we need to solve for: $x$, $y$, $v_x$, and $v_y$.  These evolve according to:

\begin{align*}
\frac{dx}{dt} &= v_x \\
\frac{dy}{dt} &= v_y \\
\frac{dv_x}{dt} &= a_x = -\frac{GM_\star x}{r^3} \\
\frac{dv_y}{dt} &= a_y = -\frac{GM_\star y}{r^3}
\end{align*}

To integrate these forward in time, we need an initial condition for each quantity.  We'll setup our system such that the Sun is at the origin (that will be one focus), and the planet begins at perihelion and orbits counterclockwise. 

![orbit_setup.png](attachment:orbit_setup.png)

The distance of perihelion from the focus is:

$$r_p = a (1 - e)$$

where $a$ is the semi-major axis and $e$ is the eccentricity.  The perihelion velocity is all in the $y$ direction and is:

$$v_y = v_p = \sqrt{\frac{GM_\star}{a} \frac{1+e}{1-e}}$$

We'll work in units of AU, years, and solar masses, in which case, $GM_\star = 4\pi^2$ (for the Sun).  

Your initial conditions should be:

  * $x(t=0) = r_p$
  * $y(t=0) = 0$
  * $v_x(t=0) = 0$
  * $v_y(t=0) = v_p$

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy import integrate
 
def rhs(t,coord):# coord = x y vx vy
    r = np.sqrt(coord[0]**2 + coord[1]**2)  # Distance from the Sun
    xdot = coord[2]
    ydot = coord[3]
    vxdot = -(4*np.pi**2 * coord[0]) / r**3
    vydot = -(4*np.pi**2 * coord[1]) / r**3
    return np.array([xdot, ydot, vxdot, vydot])

def ode_integrate(coord0,dt,tmax,method = "RK45"):
    r = integrate.solve_ivp(rhs, (0.0,tmax), coord0, method=method, dense_output=True)
    ts = np.arange(0.0,tmax,dt)
    Xs = r.sol(ts)
    return ts,Xs

#starting values
rp = 0.5  # Perihelion distance in AU
e = 0.5  # Eccentricity
a = rp / (1 - e)  # Semi-major axis
vp = np.sqrt( (4 * np.pi**2 / a) * (1 + e)  / (1 - e))  # Perihelion velocity in AU/year

x0 = rp
y0 = 0
vx0 = 0
vy0 = vp
coord0 = [x0, y0, vx0, vy0]

t, Xa = ode_integrate(coord0, 0.01,20,method = "RK45")#ten orbits
t, Xb = ode_integrate(coord0, 0.01,100, method = "DOP853")#ten orbits

# Plot the trajectory of the planet
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(Xa[0,:], Xa[1,:], label = "Runge-Kutta at 5th order, 20 revolutions")
ax.plot(Xb[0,:], Xb[1,:], label = "Runge-Kutta at 8th order, 100 revolutions") #increasing the order gives better precision, even if no constants of motion are strictly preserved and the solution eventually diverge
ax.set_title("Planetary orbit")
ax.set_xlabel("x [AU]")
ax.set_ylabel("y [AU]")
ax.grid()
ax.legend()

# Q7 Noisy signal

In [ ]:
def fdata(x, L):
    A = L/10.0
    return 2*np.sin(2*np.pi*x/L) + x*(L-x)**2/L**3 * np.cos(x) + \
           5*x*(L-x)/L**2 + A/2 + 0.1*A*np.sin(13*np.pi*x/L)

N = 2048
L = 50.0
x = np.linspace(0, L, N, endpoint=False)
orig = fdata(x, L)
print(orig)
noisy = orig + 0.5*np.random.randn(N)

In [ ]:
plt.plot(x, noisy)
plt.plot(x, orig)

In [ ]:
from scipy.signal import convolve, gaussian

sigmas = [int(N/1e3),int(N/5e2),int(N/1e2)]
smoothed = {}
for sigma in sigmas:
    gaussian_kernel = gaussian(N, sigma)
    gaussian_kernel /= np.sum(gaussian_kernel) #normalizzazion
    smoothed_data = convolve(noisy, gaussian_kernel, mode='same') 
    smoothed[str(sigma)] = smoothed_data

plt.figure(figsize=(10, 6))
plt.plot(x, noisy, label='Original Data')
for sigma, smoothed in smoothed.items():
    plt.plot(x, smoothed, label=f'Smoothed (sigma={sigma})', linestyle='-')
plt.title('Smoothing Noisy Data with Convolution')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()